## 1. Setup & Check GPU

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU! Go to Runtime → Change runtime type → GPU")

In [ ]:
!pip install -q segmentation-models-pytorch albumentations opencv-python-headless

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Configuration

In [ ]:
# ============ CONFIG ============
import os

# Dataset paths - MASKS ARE ALREADY CREATED IN segmentation2 folder
DATASET_ROOT = "/content/drive/MyDrive/segmentation2"

# Training settings
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
EPOCHS = 100
IMG_SIZE = 640
BATCH_SIZE = 8
LEARNING_RATE = 0.0001
PATIENCE = 20

# Class names
CLASS_NAMES = [
    'background',
    'Longitudinal_crack',
    'Transverse_crack',
    'Alligator_crack',
    'Pothole',
    'Other_damage'
]
NUM_CLASSES = len(CLASS_NAMES)

# Output directory
OUTPUT_DIR = "/content/unet_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Model: U-Net with {ENCODER} encoder")
print(f"Epochs: {EPOCHS}, Batch: {BATCH_SIZE}, ImgSize: {IMG_SIZE}")
print(f"Classes: {NUM_CLASSES} - {CLASS_NAMES}")

## 4. Verify Dataset Structure

In [ ]:
print("📂 Checking dataset structure...\n")

for split in ['train', 'val', 'test']:
    images_dir = os.path.join(DATASET_ROOT, split, 'images')
    masks_dir = os.path.join(DATASET_ROOT, split, 'masks')
    
    if os.path.exists(images_dir) and os.path.exists(masks_dir):
        n_images = len([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
        n_masks = len([f for f in os.listdir(masks_dir) if f.endswith('.png')])
        print(f"{split.upper()}:")
        print(f"  Images: {n_images}")
        print(f"  Masks:  {n_masks}")
        print()
    else:
        print(f"⚠️ {split.upper()}: NOT FOUND")
        print()

print("✓ Dataset verification complete!")

## 5. Create Dataset and DataLoader

In [ ]:
import torch
import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

class RoadDamageDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.images = sorted([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.images_dir, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask_name = os.path.splitext(img_name)[0] + '.png'
        mask_path = os.path.join(self.masks_dir, mask_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        
        return image, mask.long()

# Transforms
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# Create datasets and loaders
train_dataset = RoadDamageDataset(
    os.path.join(DATASET_ROOT, 'train', 'images'),
    os.path.join(DATASET_ROOT, 'train', 'masks'),
    transform=train_transform
)

val_dataset = RoadDamageDataset(
    os.path.join(DATASET_ROOT, 'val', 'images'),
    os.path.join(DATASET_ROOT, 'val', 'masks'),
    transform=val_transform
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"✓ Train: {len(train_dataset)} | Val: {len(val_dataset)}")

## 6. Build U-Net Model

In [ ]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    in_channels=3,
    classes=NUM_CLASSES,
    activation=None
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"✓ U-Net with {ENCODER} on {device}")

## 7. Training Setup

In [ ]:
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super().__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        pred = torch.softmax(pred, dim=1)
        target_one_hot = torch.nn.functional.one_hot(target, NUM_CLASSES).permute(0, 3, 1, 2).float()
        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

ce_loss = nn.CrossEntropyLoss()
dice_loss = DiceLoss()

def combined_loss(pred, target):
    return ce_loss(pred, target) + dice_loss(pred, target)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

print("✓ Loss, optimizer, scheduler ready")

## 8. Metrics Functions

In [ ]:
def calculate_iou(pred, target, num_classes):
    pred = torch.argmax(pred, dim=1)
    ious = []
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        intersection = (pred_cls & target_cls).sum().float()
        union = (pred_cls | target_cls).sum().float()
        if union > 0:
            ious.append((intersection / union).item())
    return np.mean(ious) if ious else 0.0

def calculate_pixel_accuracy(pred, target):
    pred = torch.argmax(pred, dim=1)
    correct = (pred == target).sum()
    total = target.numel()
    return (correct / total).item()

print("✓ Metrics ready")

## 9. Training Loop

In [ ]:
from tqdm import tqdm
import time

history = {'train_loss': [], 'val_loss': [], 'val_iou': [], 'val_accuracy': [], 'lr': []}
best_iou = 0.0
patience_counter = 0
best_model_path = os.path.join(OUTPUT_DIR, 'best_unet_model.pth')

print("🚀 Starting training...\n")
start_time = time.time()

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    # Training
    model.train()
    train_loss = 0.0
    for images, masks in tqdm(train_loader, desc='Training'):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = combined_loss(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Validation
    model.eval()
    val_loss = val_iou = val_accuracy = 0.0
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc='Validation'):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            val_loss += combined_loss(outputs, masks).item()
            val_iou += calculate_iou(outputs, masks, NUM_CLASSES)
            val_accuracy += calculate_pixel_accuracy(outputs, masks)
    
    val_loss /= len(val_loader)
    val_iou /= len(val_loader)
    val_accuracy /= len(val_loader)
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_iou'].append(val_iou)
    history['val_accuracy'].append(val_accuracy)
    history['lr'].append(optimizer.param_groups[0]['lr'])
    
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | mIoU: {val_iou:.4f} | Acc: {val_accuracy:.4f}")
    
    scheduler.step(val_iou)
    
    if val_iou > best_iou:
        best_iou = val_iou
        patience_counter = 0
        torch.save({'model': model.state_dict(), 'iou': best_iou, 'history': history}, best_model_path)
        print(f"✅ Best model saved! mIoU: {best_iou:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping!")
            break

print(f"\n🎉 Training done in {(time.time()-start_time)/60:.1f} min | Best mIoU: {best_iou:.4f}")

## 10. Backup to Drive

In [ ]:
import shutil
drive_backup = "/content/drive/MyDrive/unet_models"
os.makedirs(drive_backup, exist_ok=True)
shutil.copy(best_model_path, os.path.join(drive_backup, 'unet_best.pth'))
print("✓ Backed up to Drive")

## 11. Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0,0].plot(history['train_loss'], label='Train')
axes[0,0].plot(history['val_loss'], label='Val')
axes[0,0].set_title('Loss')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

axes[0,1].plot(history['val_iou'])
axes[0,1].axhline(best_iou, color='r', linestyle='--', label=f'Best: {best_iou:.4f}')
axes[0,1].set_title('mIoU')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

axes[1,0].plot(history['val_accuracy'])
axes[1,0].set_title('Accuracy')
axes[1,0].grid(True, alpha=0.3)

axes[1,1].plot(history['lr'])
axes[1,1].set_title('Learning Rate')
axes[1,1].set_yscale('log')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_curves.png'), dpi=150)
plt.show()

## 12. Test Set Evaluation

In [ ]:
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model'])
model.eval()

test_dataset = RoadDamageDataset(
    os.path.join(DATASET_ROOT, 'test', 'images'),
    os.path.join(DATASET_ROOT, 'test', 'masks'),
    transform=val_transform
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

test_iou = test_acc = 0.0
with torch.no_grad():
    for images, masks in tqdm(test_loader, desc='Testing'):
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        test_iou += calculate_iou(outputs, masks, NUM_CLASSES)
        test_acc += calculate_pixel_accuracy(outputs, masks)

test_iou /= len(test_loader)
test_acc /= len(test_loader)

print(f"\n📊 Test Results: mIoU={test_iou:.4f} | Accuracy={test_acc:.4f}")

## 13. Visualize Predictions

In [ ]:
import random

COLORS = np.array([[0,0,0], [255,0,0], [0,255,0], [0,0,255], [255,255,0], [255,0,255]], dtype=np.uint8)

test_imgs = os.listdir(os.path.join(DATASET_ROOT, 'test', 'images'))
samples = random.sample(test_imgs, min(6, len(test_imgs)))

fig, axes = plt.subplots(6, 3, figsize=(15, 24))

for idx, img_name in enumerate(samples):
    img_path = os.path.join(DATASET_ROOT, 'test', 'images', img_name)
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    mask_path = os.path.join(DATASET_ROOT, 'test', 'masks', os.path.splitext(img_name)[0] + '.png')
    gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    aug = val_transform(image=image)
    inp = aug['image'].unsqueeze(0).to(device)
    
    with torch.no_grad():
        pred = torch.argmax(model(inp), dim=1).cpu().numpy()[0]
    
    img_r = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    gt_r = cv2.resize(gt_mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
    
    axes[idx,0].imshow(img_r)
    axes[idx,0].set_title(f'Original\n{img_name}')
    axes[idx,0].axis('off')
    
    axes[idx,1].imshow(COLORS[gt_r])
    axes[idx,1].set_title('Ground Truth')
    axes[idx,1].axis('off')
    
    axes[idx,2].imshow(COLORS[pred])
    axes[idx,2].set_title('Prediction')
    axes[idx,2].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'predictions.png'), dpi=150)
plt.show()